In [1]:
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, GRU, SimpleRNN, Embedding, Conv1D, MaxPooling1D, Masking
from keras import optimizers,regularizers
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from keras.utils import plot_model

/Users/sujay/miniconda3/envs/mtp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
def parse_dataset(name):
    infile = open(name, 'r')
    lines = infile.readlines()
    data = []
    for line in lines:
        data.append([float(x) for x in line.split(',')])
    y = [x[-1] for x in data]
    data = [x[:-1] for x in data]
    return data, y

In [3]:
X_train, y_train = parse_dataset('data/estimation_without_padding.csv')
X_test, y_test = parse_dataset('data/competition_without_padding.csv')

In [4]:
def reshape_data(data):
    mat = [[] for _ in range(len(data))]
    for j in range(len(data)):
        row = data[j]
        mylist=np.empty(len(row))
        for i in range(0,len(row),2):
            mylist[i]=row[i]
            mylist[i+1]=row[i+1]
        mylist = np.reshape(mylist, (mylist.shape[0]//2,2))
        mat[j] = mylist
    return mat

In [5]:
# X_train = reshape_data(X_train)
# X_test = reshape_data(X_test)
maxlen1 = max([len(x) for x in X_train])
maxlen2 = max([len(x) for x in X_test])
X_train = pad_sequences(X_train, padding='post', value=-100, dtype=float, maxlen=maxlen1)
X_test = pad_sequences(X_test, padding='post', value=-100, dtype=float, maxlen=maxlen2)
maxlen1, maxlen2

(432, 266)

In [6]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]//2, 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]//2, 2))

In [7]:
X_train.shape, X_test.shape, X_train[0]

((1170, 216, 2), (1198, 133, 2), array([[   1. ,   -0.3],
        [   2. ,   -0.3],
        [   1. ,   -0.3],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
        [-100. , -100. ],
      

In [8]:
model = Sequential()
model.add(Masking(mask_value=-100.0, input_shape=(None, 2)))
model.add(LSTM(10, return_sequences=True, activation='relu', input_shape=(None, 2)))
# model.add(Dropout(0.3))
model.add(LSTM(10, return_sequences=False, activation='relu'))
model.add(Dense(3, activation='softmax'))

In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
plot_model(model, to_file='model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [10]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=20)

Train on 1170 samples, validate on 1198 samples
Epoch 1/20
1170/1170 [==============================] - 21s 18ms/step - loss: 1.4297 - acc: 0.3239 - val_loss: 1.4178 - val_acc: 0.3773
Epoch 2/20
1170/1170 [==============================] - 21s 18ms/step - loss: 1.1253 - acc: 0.3829 - val_loss: 1.1006 - val_acc: 0.4157
Epoch 3/20
1170/1170 [==============================] - 19s 16ms/step - loss: 1.0118 - acc: 0.4179 - val_loss: 0.9660 - val_acc: 0.4391
Epoch 4/20
1170/1170 [==============================] - 20s 17ms/step - loss: 0.9204 - acc: 0.4718 - val_loss: 0.8891 - val_acc: 0.4950
Epoch 5/20
1170/1170 [==============================] - 18s 15ms/step - loss: 0.8655 - acc: 0.5188 - val_loss: 0.8466 - val_acc: 0.5167
Epoch 6/20
1170/1170 [==============================] - 18s 15ms/step - loss: 0.8276 - acc: 0.5368 - val_loss: 0.8199 - val_acc: 0.5125
Epoch 7/20
1170/1170 [==============================] - 20s 17ms/step - loss: 0.7952 - acc: 0.5299 - val_loss: 0.8011 - val_acc: 0.5067


In [ ]:
# for seq, label in zip(X_train, y_train):
#     model.fit(np.array([seq]), [label], batch_size=1)